In [1]:
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from webdriver_manager.chrome import ChromeDriverManager

In [8]:

driver = GetDriver()
page_num = 1
url = 'https://www.linkedin.com/jobs/data-scientist-jobs-united-states?location=United%20States&pageNum=1&locationId=us%3A0&keywords=data%20scientist&position=1&start={page_num}&count=25&trk=jobs_jserp_pagination_2'.format(page_num = page_num*25)
driver.get(url)
session_ids.append(driver.session_id)
print(driver.session_id)

'



Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/will/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
a048634b20210cd24261e0455a801b5d


In [7]:
def GetDriver():
    return webdriver.Chrome(ChromeDriverManager().install())

def ScrapeJobData(job_num_on_page, driver):
    try:
        x = '/html/body/main/div[2]/div/div[2]/ul/li[{job_num_on_page}]/a'.format(job_num_on_page=job_num_on_page)
        xp = driver.find_element_by_xpath(x)
        new_url = xp.get_attribute('href')
        driver.get(new_url)
    except:
        print("site didn't match", driver.session_id)
        session_ids.append(driver.session_id)
        if len(set(session_ids)) > 1000:
            print('exceeded max number of session attempts')
            return None

        else:
            driver.close()
#             driver.delete_all_cookies()
            driver = GetDriver()
            ScrapeJobData(job_num_on_page, driver)
        
    try:
        job_title = driver.find_element_by_xpath('/html/body/main/section[3]/div[1]/section[1]/div/div[1]/h1').text
    except:
        job_title = None
    try:
        company_name = driver.find_element_by_xpath('/html/body/main/section[3]/div[1]/section[1]/div/div[1]/h3[1]/span[1]/a').text
    except:
        company_name = None
    try:
        location = driver.find_element_by_xpath('/html/body/main/section[3]/div[1]/section[1]/div/div[1]/h3[1]/span[2]').text
    except:
        location = None
    
    full_header = driver.find_element_by_xpath('/html/body/main/section[3]/div[1]/section[1]/div/div[1]').text

    job_dictionary = {'job_title': job_title
                     , 'company_name': company_name
                     , 'location': location 
                     , 'full_header': full_header
                     , 'job_posting_url' : new_url
                     }

    i = 1
    summary_data = ''
    result = ''
    while result is not None:
        try:
            summary_data = '/html/body/main/section[3]/div[1]/section[2]/ul/li[{}]'.format(i)
            result = driver.find_element_by_xpath(summary_data).text.split('\n')
            job_dictionary.update({result[0] : result[1]})
            i += 1
        except:
            result = None

    desc_1 = driver.find_element_by_xpath("/html/body/main/section[3]/div[1]/section[2]")
    job_dictionary.update({'description' : desc_1.text, 'run_date': str(datetime.datetime.now())})
    job_dictionary_df = pd.DataFrame(job_dictionary, index=[job_num_on_page])
    return job_dictionary_df

        

master_df = pd.DataFrame(columns=['job_title', 'company_name', 'location', 'Seniority level', 'Employment type', 'Job function', 'Industries', 'description', 'full_header','run_date', 'job_posting_url'])
delay = 4       
driver = GetDriver()
page_num_end = 40 
session_ids = []

for page_num in range(0, page_num_end):
    url = 'https://www.linkedin.com/jobs/data-scientist-jobs-united-states?location=United%20States&pageNum=1&locationId=us%3A0&keywords=data%20scientist&position=1&start={page_num}&count=25&trk=jobs_jserp_pagination_2'.format(page_num = page_num*25)
    driver.get(url)
    session_ids.append(driver.session_id)
    print(driver.session_id)

    for i in range(1, 25):
        master_df = pd.concat([master_df, ScrapeJobData(i, driver)], sort=True)
        driver.back()
        WebDriverWait(driver, delay)
    WebDriverWait(driver, 10)

master_df = master_df.reset_index(drop=True)
master_df.to_csv('ds_jobs_{}.csv'.format((str(datetime.datetime.today())[:10]).replace("-", "_")))






Checking for mac64 chromedriver:74.0.3729.6 in cache
Driver found in /Users/will/.wdm/chromedriver/74.0.3729.6/mac64/chromedriver
a5e044730599341d0155225db04f0445
site didn't match a5e044730599341d0155225db04f0445


Exception ignored in: <function Service.__del__ at 0x1154c7a60>
Traceback (most recent call last):
  File "/Users/will/anaconda3/lib/python3.7/site-packages/selenium/webdriver/common/service.py", line 176, in __del__
    self.stop()
  File "/Users/will/anaconda3/lib/python3.7/site-packages/selenium/webdriver/common/service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "/Users/will/anaconda3/lib/python3.7/site-packages/selenium/webdriver/common/service.py", line 127, in send_remote_shutdown_command
    url_request.urlopen("%s/shutdown" % self.service_url)
  File "/Users/will/anaconda3/lib/python3.7/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/Users/will/anaconda3/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/Users/will/anaconda3/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/Users/will/anaconda3/lib/python3.7/urllib/request.py", lin

Traceback (most recent call last):
  File "<ipython-input-7-7a1b4907125f>", line 7, in ScrapeJobData
    xp = driver.find_element_by_xpath(x)
  File "/Users/will/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "/Users/will/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 978, in find_element
    'value': value})['value']
  File "/Users/will/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/Users/will/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/div[2]/div/d

  File "/Users/will/anaconda3/lib/python3.7/site-packages/pkg_resources/__init__.py", line 1960, in <genexpr>
    if dist_factory(path_item, entry, only)
  File "/Users/will/anaconda3/lib/python3.7/site-packages/pkg_resources/__init__.py", line 1977, in dist_factory
    is_meta = any(map(lower.endswith, ('.egg-info', '.dist-info')))
KeyboardInterrupt


TypeError: can only concatenate str (not "list") to str

In [6]:
master_df

,Employment type,Industries,Job function,Seniority level,company_name,description,full_header,job_posting_url,job_title,location,run_date
1,Full-time,Entertainment,Other,Mid-Senior level,VelvetJobs,Job description\nData Scientist - League of Le...,"Data Scientist\nVelvetJobsLos Angeles, CA, US\...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Los Angeles, CA, US",2019-05-21 06:24:24.846196
2,Full-time,Computer Software,EngineeringInformation Technology,Entry level,Eversight,Job description\nThe Opportunity\n\nEversight ...,"Data Scientist\nEversightPalo Alto, CA, US\n1 ...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Palo Alto, CA, US",2019-05-21 06:24:26.888452
3,Full-time,Information Technology and ServicesComputer So...,EngineeringInformation Technology,Entry level,Apex Life Sciences,Job description\nJob Description\nJob #: 9525...,"Data Scientist\nApex Life SciencesSugar Land, ...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Sugar Land, TX, US",2019-05-21 06:24:28.554995
4,Full-time,Internet,EngineeringInformation Technology,Entry level,Bixal,Job description\nWho We Are\n\nBixal uses stra...,"Data Scientist\nBixalWashington, D.C., DC, US\...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Washington, D.C., DC, US",2019-05-21 06:24:30.234788
5,Full-time,Marketing and AdvertisingComputer SoftwareInte...,EngineeringInformation Technology,Entry level,Origin Multilingual -Europe's Leading Language...,"Job description\nBeeswax, named by Business In...",Data Scientist\nOrigin Multilingual -Europe's ...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"New York City, NY, US",2019-05-21 06:24:32.020498
6,Full-time,Consumer ElectronicsInternetResearch,EngineeringInformation TechnologyResearch,Mid-Senior level,Samsung Electronics America,Job description\nData Scientist (Home Applianc...,Data Scientist\nSamsung Electronics AmericaPla...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Plano, Texas",2019-05-21 06:24:33.703668
7,Full-time,Information Technology and Services,Information Technology,Mid-Senior level,zData Inc. - an atos business,Job description\nzData Inc is taking Big Data ...,Big Data DevOps Engineer\nzData Inc. - an atos...,https://www.linkedin.com/jobs/view/big-data-de...,Big Data DevOps Engineer,"East Rutherford, New Jersey",2019-05-21 06:24:35.558515
8,Full-time,Information Technology and ServicesComputer So...,EngineeringInformation Technology,Entry level,Apex Systems,Job description\nJob Description\n\nJob #: 952...,"Data Scientist\nApex SystemsSugar Land, TX, US...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Sugar Land, TX, US",2019-05-21 06:24:37.274049
9,Full-time,Marketing and Advertising,MarketingScienceInformation Technology,Mid-Senior level,Branded Entertainment Network,Job description\nWho YOU Are\nDo you instincti...,Senior Data Scientist\nBranded Entertainment N...,https://www.linkedin.com/jobs/view/senior-data...,Senior Data Scientist,"Provo, Utah",2019-05-21 06:24:38.879946
10,Full-time,Information Technology and ServicesComputer So...,EngineeringInformation Technology,Entry level,Dovel Technologies,"Job description\nDovel Technologies, LLC provi...","Data Scientist\nDovel TechnologiesMcLean, VA, ...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"McLean, VA, US",2019-05-21 06:24:40.584938


In [5]:
page_num

16

In [17]:
master_df

,Employment type,Industries,Job function,Seniority level,company_name,description,full_header,job_posting_url,job_title,location,run_date
1,Full-time,"Computer SoftwareHospital & Health CareHealth,...",Information Technology,Mid-Senior level,Carrot Health,Job description\nCarrot Health is looking for ...,Senior Data Scientist\nCarrot HealthGreater Mi...,https://www.linkedin.com/jobs/view/senior-data...,Senior Data Scientist,Greater Minneapolis-St. Paul Area,2019-05-14 05:59:26.615554
2,Full-time,Information Technology and Services,EngineeringInformation Technology,Mid-Senior level,Quantum Workplace,Job description\nMake work better every day\nQ...,"Data Scientist\nQuantum WorkplaceOmaha, Nebras...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Omaha, Nebraska",2019-05-14 05:59:28.481720
3,Full-time,NaN,NaN,Not Applicable,"Olaris, Inc","Job description\nIf you like diving into data,...","Data Scientist\nOlaris, IncGreater Boston Area...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,Greater Boston Area,2019-05-14 05:59:30.139451
4,Contract,NaN,NaN,Not Applicable,None,Job description\nThe Birnbaum Women’s Leadersh...,Data Scientist\nNew York University School of ...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,Greater New York City Area,2019-05-14 05:59:32.014572
5,Full-time,Computer SoftwareInternet,EngineeringInformation Technology,Mid-Senior level,cohesion,Job description\nPassionate about digitally tr...,"Data Scientist\ncohesionChicago, Illinois\nNew...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Chicago, Illinois",2019-05-14 05:59:33.826795
6,Full-time,Biotechnology,Information TechnologyAnalystEngineering,Mid-Senior level,Convergent Genomics,Job description\nAbout Convergent Genomics\nCo...,Data Scientist\nConvergent Genomics San Franci...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,San Francisco Bay Area,2019-05-14 05:59:35.697707
7,Full-time,Financial Services,EngineeringAnalyst,Mid-Senior level,Cypress HCM,Job description\nData Scientist\n Cypress is ...,"Data Scientist\nCypress HCMSan Diego, Californ...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"San Diego, California",2019-05-14 05:59:37.452065
8,Full-time,Information Technology and ServicesInsuranceFi...,EngineeringInformation Technology,Entry level,Crum & Forster,Job description\nWho We Are\n\nCrum & Forster ...,"Associate Data Scientist\nCrum & ForsterAkron,...",https://www.linkedin.com/jobs/view/associate-d...,Associate Data Scientist,"Akron, OH, US",2019-05-14 05:59:39.064320
9,Full-time,Retail,AnalystScienceResearch,Mid-Senior level,Walmart,Job description\nStaff Data Scientist\n\nThis ...,"Staff Data Scientist\nWalmartFayetteville, Ark...",https://www.linkedin.com/jobs/view/staff-data-...,Staff Data Scientist,"Fayetteville, Arkansas Area",2019-05-14 05:59:40.854109
10,Full-time,Investment Management,Finance,Associate,Selby Jennings,"Job description\nMy client, a well regarded gl...",Data Scientist (NLP/Deep Learning)\nSelby Jenn...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist (NLP/Deep Learning),"San Francisco, California, United States",2019-05-14 05:59:42.915527


In [54]:
master_df_2 = master_df

In [47]:
master_df_1 = master_df

In [60]:
master_df = pd.concat([master_df, master_df_1])

In [61]:
master_df = pd.concat([master_df, master_df_2])

In [64]:
master_df = master_df.reset_index(drop=True)

In [65]:
master_df

,Employment type,Industries,Job function,Seniority level,company_name,description,full_header,job_posting_url,job_title,location,run_date
0,Full-time,Information Technology and ServicesComputer So...,EngineeringInformation Technology,Mid-Senior level,IBM,Job description\nJob Description\n\nIn this ne...,"Data Scientist\nIBMNew York City, NY, US\n1 we...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"New York City, NY, US",2019-05-05 08:25:43.363819
1,Full-time,InsuranceFinancial ServicesHospital & Health Care,EngineeringInformation Technology,Entry level,"Aetna, a CVS Health Company",Job description\nJob Description\n\nPOSITION S...,"Data Scientist\nAetna, a CVS Health CompanyNew...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"New York City, NY, US",2019-05-05 08:25:45.181828
2,Full-time,InsuranceFinancial ServicesHospital & Health Care,EngineeringInformation Technology,Entry level,"Aetna, a CVS Health Company",Job description\nJob Description\n\nPOSITION S...,"Data Scientist\nAetna, a CVS Health CompanyNew...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"New York City, NY, US",2019-05-05 08:25:46.789826
3,Full-time,InsuranceFinancial ServicesHospital & Health Care,EngineeringInformation Technology,Entry level,"Aetna, a CVS Health Company",Job description\nJob Description\n\nPOSITION S...,"Data Scientist\nAetna, a CVS Health CompanyNew...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"New York City, NY, US",2019-05-05 08:25:48.538865
4,Full-time,Information Technology and ServicesResearchCom...,Information TechnologyEngineeringResearch,Mid-Senior level,Big Cloud,Job description\nHow quickly would humanity pr...,"Data Scientist\nBig CloudWestport, Connecticut...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Westport, Connecticut",2019-05-05 08:25:50.318100
5,Full-time,Information Technology and ServicesResearchCom...,Information TechnologyEngineeringResearch,Mid-Senior level,Big Cloud,Job description\nAre you a US citizen or Perma...,"Data Scientist\nBig CloudRedwood City, Califor...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Redwood City, California",2019-05-05 08:25:51.996287
6,Full-time,Information Technology and ServicesResearchCom...,Information TechnologyEngineeringResearch,Mid-Senior level,Big Cloud,Job description\nAre you a US citizen or Perma...,Data Scientist\nBig CloudWashington D.C. Metro...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,Washington D.C. Metro Area,2019-05-05 08:25:53.546403
7,Full-time,Information Technology and ServicesComputer So...,EngineeringInformation Technology,Entry level,Amiga Informatics,"Job description\nJob Description Hello, Trust ...",Data Scientist(data analyst+data engineer)\nAm...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist(data analyst+data engineer),"Alviso Ridge, CA, US",2019-05-05 08:25:54.998872
8,Full-time,Information Technology and ServicesResearchCom...,Information TechnologyEngineeringResearch,Mid-Senior level,Big Cloud,Job description\nAre you a US citizen or Perma...,Data Scientist\nBig CloudWashington D.C. Metro...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,Washington D.C. Metro Area,2019-05-05 08:25:56.572308
9,Full-time,Marketing and AdvertisingComputer SoftwareInte...,Information Technology,Associate,None,"Job description\nAt Plato, we’re on a mission ...",Data Analyst / Scientist [Full-time]\nPlatoSan...,https://www.linkedin.com/jobs/view/data-analys...,Data Analyst / Scientist [Full-time],"San Francisco, CA, US",2019-05-05 08:25:58.154154


In [8]:
master_df.dtypes

Employment type    object
Industries         object
Job function       object
Seniority level    object
company_name       object
description        object
full_header        object
job_posting_url    object
job_title          object
location           object
run_date           object
dtype: object

In [105]:
master_df['python'] = master_df['description'].apply(lambda x: 'python' in x.lower())

In [66]:
# master_df.to_csv('ds_jobs.csv')

In [15]:
df = pd.DataFrame.from_csv("ds_jobs.csv")

/Users/will/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [17]:
df.dtypes

employment_type    object
industries         object
job function       object
seniority_level    object
company_name       object
description        object
full_header        object
job_posting_url    object
job_title          object
location           object
run_date           object
dtype: object

In [18]:
df

,employment_type,industries,job function,seniority_level,company_name,description,full_header,job_posting_url,job_title,location,run_date
0,Full-time,Information Technology and ServicesComputer So...,EngineeringInformation Technology,Mid-Senior level,IBM,Job description\nJob Description\n\nIn this ne...,"Data Scientist\nIBMNew York City, NY, US\n1 we...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"New York City, NY, US",2019-05-05 08:25:43.363819
1,Full-time,InsuranceFinancial ServicesHospital & Health Care,EngineeringInformation Technology,Entry level,"Aetna, a CVS Health Company",Job description\nJob Description\n\nPOSITION S...,"Data Scientist\nAetna, a CVS Health CompanyNew...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"New York City, NY, US",2019-05-05 08:25:45.181828
2,Full-time,InsuranceFinancial ServicesHospital & Health Care,EngineeringInformation Technology,Entry level,"Aetna, a CVS Health Company",Job description\nJob Description\n\nPOSITION S...,"Data Scientist\nAetna, a CVS Health CompanyNew...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"New York City, NY, US",2019-05-05 08:25:46.789826
3,Full-time,InsuranceFinancial ServicesHospital & Health Care,EngineeringInformation Technology,Entry level,"Aetna, a CVS Health Company",Job description\nJob Description\n\nPOSITION S...,"Data Scientist\nAetna, a CVS Health CompanyNew...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"New York City, NY, US",2019-05-05 08:25:48.538865
4,Full-time,Information Technology and ServicesResearchCom...,Information TechnologyEngineeringResearch,Mid-Senior level,Big Cloud,Job description\nHow quickly would humanity pr...,"Data Scientist\nBig CloudWestport, Connecticut...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Westport, Connecticut",2019-05-05 08:25:50.318100
5,Full-time,Information Technology and ServicesResearchCom...,Information TechnologyEngineeringResearch,Mid-Senior level,Big Cloud,Job description\nAre you a US citizen or Perma...,"Data Scientist\nBig CloudRedwood City, Califor...",https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,"Redwood City, California",2019-05-05 08:25:51.996287
6,Full-time,Information Technology and ServicesResearchCom...,Information TechnologyEngineeringResearch,Mid-Senior level,Big Cloud,Job description\nAre you a US citizen or Perma...,Data Scientist\nBig CloudWashington D.C. Metro...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,Washington D.C. Metro Area,2019-05-05 08:25:53.546403
7,Full-time,Information Technology and ServicesComputer So...,EngineeringInformation Technology,Entry level,Amiga Informatics,"Job description\nJob Description Hello, Trust ...",Data Scientist(data analyst+data engineer)\nAm...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist(data analyst+data engineer),"Alviso Ridge, CA, US",2019-05-05 08:25:54.998872
8,Full-time,Information Technology and ServicesResearchCom...,Information TechnologyEngineeringResearch,Mid-Senior level,Big Cloud,Job description\nAre you a US citizen or Perma...,Data Scientist\nBig CloudWashington D.C. Metro...,https://www.linkedin.com/jobs/view/data-scient...,Data Scientist,Washington D.C. Metro Area,2019-05-05 08:25:56.572308
9,Full-time,Marketing and AdvertisingComputer SoftwareInte...,Information Technology,Associate,NaN,"Job description\nAt Plato, we’re on a mission ...",Data Analyst / Scientist [Full-time]\nPlatoSan...,https://www.linkedin.com/jobs/view/data-analys...,Data Analyst / Scientist [Full-time],"San Francisco, CA, US",2019-05-05 08:25:58.154154


In [75]:
df.groupby('company_name').company_name.count().sort_values(ascending=False)

company_name
Big Cloud                                       6
O'Reilly Media                                  5
Aetna, a CVS Health Company                     4
Bravo Consulting                                4
Amiga Informatics                               4
Verizon                                         4
Capital One                                     4
Xometry                                         3
Infosys Public Services                         3
Infosys                                         3
Proofpoint                                      3
Bethesda Softworks                              3
Kwai Inc.                                       3
Primus Software Corporation                     3
Mastech                                         3
Apex Life Sciences                              3
Apex Systems                                    3
PADI                                            3
BIND                                            3
WeWork                               

In [8]:
master_df.groupby('location').location.count().sort_values(ascending=False)

location
New York City, NY, US                                         6
New York, New York                                            5
San Francisco, CA, US                                         5
Seattle, WA, US                                               3
Austin, TX, US                                                3
Chicago, IL, US                                               3
Washington, D.C., DC, US                                      3
San Francisco Bay Area                                        3
Atlanta, GA, US                                               2
Greater Atlanta Area                                          2
South San Francisco, CA, US                                   2
San Francisco, California, United States                      2
San Francisco, California                                     2
Sunnyvale, CA, US                                             2
New York, New York, United States                             2
Arlington, VA, US              

In [88]:
# driver = webdriver.Chrome(executable_path=r'/usr/local/bin/chromedriver')
# url = 'https://www.linkedin.com/jobs/data-scientist-jobs-united-states?location=United%20States&pageNum=1&locationId=us%3A0&keywords=data%20scientist&position=1&start={page_num}&count=25&trk=jobs_jserp_pagination_2'.format(page_num = 0*25)
# driver.get(url)


In [89]:
ScrapeJobData(1)

In [78]:
len(master_df)

0

In [25]:
driver.back()

In [10]:
job_dictionary.keys()

dict_keys(['job_title', 'company_name', 'location', 'Seniority level', 'Employment type', 'Job function', 'Industries', 'description'])

In [11]:
master_df

,job_title,company_name,location,Seniority level,Employment type,Job function,Industries,description


,job_title,company_name,location,Seniority level,Employment type,Job function,Industries,description
0,Data Scientist,The Select Group,"Charlotte, NC",Mid-Senior level,Contract,Information Technology,Computer NetworkingInformation Technology and ...,Job description\nDATA SCIENTIST\nAn industry l...
